In this notebook, we check if a prime knots of 20 crossings has the same zero-surgery as T(5,2). For that we first load the list of 20 crossing knots that have the same Alexander polynomial as T(5,2). Then we check which of these have hyperbolic 0-surgeries. For the ones that are not hyperbolic we use regina to determine its type.

In [1]:
import snappy
import regina
import csv
import time

def all_positive(manifold):
    '''
    Checks if the solution type of a triangulation is positive.
    '''
    return manifold.solution_type() == 'all tetrahedra positively oriented'

def find_positive_triangulations(manifold,number=1,tries=100):
    '''
    Searches for one triangulation with a positive solution type.
    (Or if number is set to a different value also for different such triangulations.)
    '''
    M = manifold.copy()
    pos_triangulations=[]
    for i in range(tries):
        if all_positive(M):
            pos_triangulations.append(M)
            if len(pos_triangulations)==number:
                return pos_triangulations
            break
        M.randomize()
    for d in M.dual_curves(max_segments=500):
        X = M.drill(d)
        X = X.filled_triangulation()
        X.dehn_fill((1,0),-1)
        for i in range(tries):
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
                break
            X.randomize()

    # In the closed case, here is another trick.
    if all(not c for c in M.cusp_info('is_complete')):
        for i in range(tries):
            # Drills out a random edge
            X = M.__class__(M.filled_triangulation())
            if all_positive(X):
                pos_triangulations.append(X)
                if len(pos_triangulations)==number:
                    return pos_triangulations
            break
            M.randomize()
    return pos_triangulations

def better_volume(M,index=100,try_hard=False):
    '''Computes the verified volume. Returns 0 if SnapPy could not do it.'''
    count=0
    while count<index:
        try:
            return M.volume(verified=True)
        except:
            M.randomize()
            count=count+1
    if try_hard==True:
        pos_triang=find_positive_triangulations(M,number=1,tries=index)
        for X in pos_triang:
            vol=better_volume(X,index)
            if vol!=0:
                return vol
    return 0

def change_notation(dt_code):
    """
    Changes Dowker-Thistlewait notation from alphabetical to numerical
    Input:
        dt_code (string): alphabetical DT notation
    Return:
        (string): numerical DT notation
    """
    alpha = "abcdefghijklmnopqrstuvwxyz"
    Alpha = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
    result = []
    for letter in dt_code:
        if letter in alpha:
            result.append(2* (alpha.index(letter) + 1))
        elif letter in Alpha:
            result.append(-2 * (Alpha.index(letter) + 1))
        else:
            print(dt_code)
    return "DT: " + str([tuple(result)])

def fill_triangulation(M):
    '''
    Fills all cusps but one.
    '''
    if M.num_cusps()==1:
        return M
    M=M.filled_triangulation([0])
    M=fill_triangulation(M)
    return M

In [6]:
possible_same_0_surgery=[] 
with open('nonalt_hyp_20_matches.csv', 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                possible_same_0_surgery.append(row[0])

In [7]:
len(possible_same_0_surgery)

18883

In [20]:
start_time = time.time()
maybe_not_hyperbolic=[]

for DT in possible_same_0_surgery:
        K=snappy.Link('DT: '+DT).exterior()
        K.dehn_fill((0,1))
        vol=better_volume(K)
        if vol==0:
            maybe_not_hyperbolic.append([knot,cros,DT])
print('Number of knots whose zero surgery appears to not be hyperbolic:',len(maybe_not_hyperbolic))
print('Time taken: %s hours ' % ((time.time() - start_time)/3600))

Number of knots whose zero surgery appears to not be hyperbolic: 0
Time taken: 0.28341284586323634 hours 


In [21]:
possible_same_0_surgery=[] 
with open('nonalt_hyp_20_matches_1.csv', 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                possible_same_0_surgery.append(row[0])
                
len(possible_same_0_surgery)

23608

In [22]:
start_time = time.time()

for DT in possible_same_0_surgery:
        K=snappy.Link('DT: '+DT).exterior()
        K.dehn_fill((0,1))
        vol=better_volume(K)
        if vol==0:
            maybe_not_hyperbolic.append([knot,cros,DT])
print('Number of knots whose zero surgery appears to not be hyperbolic:',len(maybe_not_hyperbolic))
print('Time taken: %s hours ' % ((time.time() - start_time)/3600))

Number of knots whose zero surgery appears to not be hyperbolic: 0
Time taken: 0.35149660309155784 hours 


For the non-hyperbolic knot that have the same Alexander polynomial we use the number of subgroups of a fixed order as invariant.

In [61]:
possible_same_0_surgery=[] 
with open('nonhyp_20_matches_1.csv', 'r') as file:
            reader = csv.reader(file)
            for row in reader:
                possible_same_0_surgery.append(row[0])
                
possible_same_0_surgery

['tatcEfkmIqrBaNToJdshgpL']

In [62]:
K=snappy.Manifold('K5a2(0,1)')
L=snappy.Link('DT: '+'tatcEfkmIqrBaNToJdshgpL').exterior()
L.dehn_fill((0,1))

In [63]:
def subgroups_of_order_n(M,n):
    '''
    Returns the number of subgroups of order n of the fundamental group of M.
    Warning: Works only fast for small n.
    '''
    return len(snappy.Manifold(M).covers(n))

def subgroups_up_to_order_k(M,k):
    '''
    Returns the vector containing the number of subgroups of order n of the fundamental group of M for n=2,...,k.
    Warning: Works only fast for small k.
    '''
    vector=[]
    for n in range(2,k+1):
        vector.append(subgroups_of_order_n(M,n))
    return vector

In [64]:
subgroups_up_to_order_k(K,7)

[1, 1, 1, 3, 4, 2]

In [66]:
subgroups_up_to_order_k(L,7)

[1, 1, 1, 9, 16, 11]

This shows that the (5,2)-torus knots has no friend with at most 20 crossings.